In [21]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

current_directory = os.listdir()
match = [match for match in current_directory if 'for_modeling' in match]
match = match[0]


df = pd.read_csv(match, index_col=0)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99178 entries, 0 to 99177
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               99178 non-null  object 
 1   LittleFallsGage    99174 non-null  float64
 2   01646500-00060     99174 non-null  float64
 3   SenecaGage         99178 non-null  float64
 4   01645000-00060     99178 non-null  float64
 5   PointofRocksGage   99178 non-null  float64
 6   01638500-00060     99178 non-null  int64  
 7   EdwardsGage        99178 non-null  float64
 8   ShhepardstownGage  99177 non-null  float64
 9   01618000-00060     99177 non-null  float64
 10  HancockGage        99178 non-null  float64
 11  01613000-00060     99178 non-null  int64  
 12  SpringfieldGage    99178 non-null  float64
 13  01608500-00060     99178 non-null  int64  
dtypes: float64(10), int64(3), object(1)
memory usage: 11.4+ MB


In [23]:
# lag variables
# train, test split

df['PointofRocksGage'] = df['PointofRocksGage'].shift(16)
df['EdwardsGage'] = df['EdwardsGage'].shift(12)
df['ShhepardstownGage'] = df['ShhepardstownGage'].shift(32)
df['HancockGage'] = df['HancockGage'].shift(60)
df['SpringfieldGage'] = df['SpringfieldGage'].shift(68)

df.drop(df.columns[[2,4,6,9,11,13]], axis=1, inplace=True)

In [24]:
df

,Date,LittleFallsGage,SenecaGage,PointofRocksGage,EdwardsGage,ShhepardstownGage,HancockGage,SpringfieldGage
0,2022-04-12 13:15:00,5.26,2.39,NaN,NaN,NaN,NaN,NaN
1,2022-04-12 13:30:00,5.25,2.40,NaN,NaN,NaN,NaN,NaN
2,2022-04-12 13:45:00,5.25,2.40,NaN,NaN,NaN,NaN,NaN
3,2022-04-12 14:00:00,5.24,2.40,NaN,NaN,NaN,NaN,NaN
4,2022-04-12 14:15:00,5.24,2.40,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
99173,2023-04-12 12:00:00,3.51,2.05,2.36,4.50,3.21,4.52,2.3
99174,2023-04-12 12:15:00,3.52,2.06,2.37,4.50,3.20,4.52,2.3
99175,2023-04-12 12:30:00,3.50,2.07,2.37,4.49,3.20,4.52,2.3
99176,2023-04-12 12:45:00,3.52,2.08,2.36,4.49,3.20,4.52,2.3


In [30]:
df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(train.index)

In [35]:
Y_train = df_train['LittleFallsGage']
X_train = df_train.drop(columns=['Date', 'LittleFallsGage'])

In [ ]:
# need to break them out by seperate hourly predictions


In [36]:
model_17hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation = 'relu', input_shape = (1,)),  # input shape required
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

model_17hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

In [37]:
# Train
model_17hrs.fit(X_train, Y_train, epochs=150, batch_size=32, verbose=0)

ValueError: in user code:

    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/envs/RiverDAta/lib/python3.8/site-packages/keras/engine/input_spec.py", li

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))